In [49]:
import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial
from manipulation import FindResource

from manipulation.station import load_scenario, MakeHardwareStation

from pydrake.multibody.plant import AddMultibodyPlantSceneGraph

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad, MakeManipulationStation, AddRgbdSensors
from manipulation.utils import running_as_notebook, ConfigureParser

In [28]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7004


In [52]:
def CreateIiwaControllerPlant():
    """creates plant that includes only the robot and gripper, used for controllers."""
    robot_sdf_path = "package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf"
    gripper_sdf_path = "package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
    sim_timestep = 1e-3
    plant_robot = MultibodyPlant(sim_timestep)
    parser = Parser(plant=plant_robot)
    parser.AddModelsFromUrl(robot_sdf_path)
    parser.AddModelsFromUrl(gripper_sdf_path)
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.world_frame(),
        frame_on_child_M=plant_robot.GetFrameByName("iiwa_link_0"),
    )
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.GetFrameByName("iiwa_link_7"),
        frame_on_child_M=plant_robot.GetFrameByName("body"),
        X_FM=RigidTransform(
            RollPitchYaw(np.pi / 2, 0, np.pi / 2), np.array([0, 0, 0.114])
        ),
    )
    plant_robot.mutable_gravity_field().set_gravity_vector([0, 0, 0])
    plant_robot.Finalize()

    link_frame_indices = []
    for i in range(8):
        link_frame_indices.append(
            plant_robot.GetFrameByName("iiwa_link_" + str(i)).index()
        )

    return plant_robot, link_frame_indices

def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
    """
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            filename="file:///workspaces/manip-final-project/withcamera.scenarios.yaml",
            time_step=1e-3,
        )
    )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
    g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    builder.Connect(
        q_traj_system.get_output_port(), station.GetInputPort("iiwa_position")
    )
    builder.Connect(
        g_traj_system.get_output_port(), station.GetInputPort("wsg_position")
    )

    diagram = builder.Build()

    simulator = Simulator(diagram)
    visualizer.StartRecording(False)
    simulator.AdvanceTo(duration)
    visualizer.PublishRecording()

    return simulator, plant

def setup_manipulation_station():
    scenario = load_scenario(
        filename = "/workspaces/manip-final-project/withcamera.scenarios.yaml",
        scenario_name = "Tennis"
    )
    
    builder = DiagramBuilder()
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat = meshcat))
    # station = builder.AddSystem(
    #     MakeManipulationStation(
    #         filename="file:///workspaces/manip-final-project/withcamera.scenarios.yaml",
    #         time_step=1e-3,
    #     )
    # )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
    builder.Connect(
        wsg_position.get_output_port(), station.GetInputPort("wsg_position")
    )

    diagram = builder.Build()

    context = plant.CreateDefaultContext()
    gripper = plant.GetBodyByName("body")

    initial_pose = plant.EvalBodyPoseInWorld(context, gripper)

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.01)

    return initial_pose

initial_pose = setup_manipulation_station()

RuntimeError: GetModelInstanceByName(): There is no model instance named 'camera0'. The current model instances are 'DefaultModelInstance', 'WorldModelInstance', 'bottle', 'iiwa', 'wsg'.

In [31]:
print("scene system")
environment_diagram, environment_context = SceneSystem()
print("scene system")
cameras = []
print("appending")
cameras.append(
    CameraSystem(0, meshcat, environment_diagram, environment_context)
)
cameras.append(
    CameraSystem(1, meshcat, environment_diagram, environment_context)
)

# plt.imshow(cameras[0].rgb_im)
# plt.title("View from camera 0")
# plt.show()

# plt.imshow(cameras[1].rgb_im)
# plt.title("View from camera 1")
# plt.show()

In [ ]:
rpy = RollPitchYaw([np.pi, 0, np.pi/2])
goal_rotation = RotationMatrix(rpy)
goal_position = np.array([0.95, 0, 0.23])
goal_pose = RigidTransform(goal_rotation, goal_position)

up_position = np.array([0.95, 0, 1.0])
up_pose = RigidTransform(goal_rotation, up_position)

## Interpolate Pose for entry.
def make_gripper_orientation_trajectory(prev_pose, pose):
    traj = PiecewiseQuaternionSlerp()
    traj.Append(0.0, prev_pose.rotation())
    traj.Append(5.0, pose.rotation())
    return traj

def make_gripper_position_trajectory(prev_pose, pose):
    traj = PiecewisePolynomial.FirstOrderHold(
        [0.0, 5.0],
        np.vstack(
            [
                [prev_pose.translation()],
                [pose.translation()],
            ]
        ).T,
    )
    return traj

entry_traj_rotation = make_gripper_orientation_trajectory(initial_pose, goal_pose)
entry_traj_translation = make_gripper_position_trajectory(initial_pose, goal_pose)
up_traj_rotation = make_gripper_orientation_trajectory(goal_pose, up_pose)
up_traj_translation = make_gripper_position_trajectory(goal_pose, up_pose)

def InterpolatePosePickup(t):
    if t < 5: 
        return RigidTransform(
            RotationMatrix(entry_traj_rotation.orientation(t)),
            entry_traj_translation.value(t),
        )
    elif t >= 5 and t < 6:
        return RigidTransform(
            RotationMatrix(entry_traj_rotation.orientation(5.0)),
            entry_traj_translation.value(5.0),
        )
    else: 
        return RigidTransform(
            RotationMatrix(up_traj_rotation.orientation((t - 6)/5)),
            up_traj_translation.value((t - 6)/5),
        )

t_lst = np.linspace(0, 11, 10)
pose_lst = []
for t in t_lst:
    AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePosePickup(t), opacity=0.2)
    pose_lst.append(InterpolatePosePickup(t))

# Create gripper trajectory.
gripper_t_lst = np.array([0.0, 5.0, 6.0, 11.0])
gripper_knots = np.array([0.11, 0.11, 0.05, 0.0]).reshape(1, 4)
g_traj = PiecewisePolynomial.FirstOrderHold(gripper_t_lst, gripper_knots)

def create_q_knots(pose_lst):
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
    contain gripper joints, but these should not matter to the constraints.
    @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    plant, _ = CreateIiwaControllerPlant()
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("body")
    q_nominal = np.array(
        [0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0, 0.0, 0.0]
    )  # nominal joint angles for joint-centering.

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=R_WG,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=bounds,
        )

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame,
            frameB=gripper_frame,
            p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower,
            p_AQ_upper=p_WG_upper,
        )

    for i in range(len(pose_lst)):
        ik = inverse_kinematics.InverseKinematics(plant)
        q_variables = ik.q()  # Get variables for MathematicalProgram
        prog = ik.prog()  # Get MathematicalProgram

        #### Modify here ###############################

        if i == 0:
            prog.SetInitialGuess(q_variables, q_nominal)
        else:
            prog.SetInitialGuess(q_variables, q_knots[i - 1])

        x = q_variables - q_nominal
        prog.AddCost(x.dot(x))
        X_WG = pose_lst[i]
        p_WG = X_WG.translation()
        z_epsilon = 0.01
        theta_epsilon = 0.05
        p_WG_lower = np.array([p_WG[0], p_WG[1], p_WG[2] - z_epsilon])
        p_WG_upper = np.array([p_WG[0], p_WG[1], p_WG[2] + z_epsilon])
        R_WG = X_WG.rotation()
        AddPositionConstraint(ik, p_WG_lower, p_WG_upper)
        AddOrientationConstraint(ik, R_WG, theta_epsilon)

        ################################################

        result = Solve(prog)

        assert result.is_success()

        q_knots.append(result.GetSolution(q_variables))

    return q_knots

In [ ]:
q_knots = np.array(create_q_knots(pose_lst))
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)
simulator, station_plant = BuildAndSimulateTrajectory(q_traj, g_traj, 11.0)